In [2]:
import glob

import pandas as pd

import scib
import scanpy as sc

from data_manage import *

/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/torchvision/transforms/functional_pil.py:228: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/torchvision/transforms/functional_pil.py:295: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/torchvision/transforms/functional_pil.py:311: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/torchvision/transforms/functional_pil.py:328: DeprecationWarning: BI

## Raw data

In [7]:
import scanorama

In [9]:
path = '../../data/raw_data/human_pancreas_norm_complexBatch.h5ad'
adata = sc.read(path)

/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/X' was written without encoding metadata.
  d[k] = read_elem(f[k])
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/layers/counts' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/tech' was written without encoding metadata.
  categories = read_elem(categories_dset)
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/t

In [13]:

# Integration.
scanorama.integrate_scanpy(adata, dimred = 5)

# Batch correction.
corrected = scanorama.correct_scanpy(adata)

# Integration and batch correction.
corrected = scanorama.correct_scanpy(adatas, return_dimred=True)

Found 19093 genes among all datasets


KeyboardInterrupt: 

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation()
adata.obsm["X_emb"] = adata.obsm["X_scVI"]

In [ ]:
from scib.metrics.silhouette import silhouette_batch, silhouette
from scib.metrics.lisi import lisi_graph
s_batch = silhouette_batch(adata, "tech", "celltype", "X_emb")
s = silhouette(adata, "celltype", "X_emb")

In [ ]:
for path in ['../../data/raw_data/human_pancreas_norm_complexBatch.h5ad']:
    adata = sc.read(path)
    adata_raw = sc.read(path)
    c_type, n_cell, cell_times, seq_depth = "all", 0, 0, 0
    # train scgen
    for n_latent in [5, 50, 100]: 
        fig_path = "scvi_immune_n_latent_" + str(n_latent)
        model_path ="../../results/scvi_models/" + "immune_" 
        
        sc.pp.highly_variable_genes(
            adata,
            flavor = "seurat_v3",
            n_top_genes = 2000,
            batch_key = "tech",
            subset = True,
            layer="counts"
        )
        scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="tech")
        vae = scvi.model.SCVI(adata, n_layers = 2, n_latent= n_latent, gene_likelihood="nb")
        lvae = scvi.model.SCANVI.from_scvi_model(
            vae,
            adata=adata,
            labels_key="celltype",
            unlabeled_category="Unknown",
        )
        lvae.train(max_epochs= 1, n_samples_per_label=100)


        
        pcr = scib.me.pcr_comparison(adata_pre = adata_raw, adata_post = adata, covariate = "celltype")
        nmi = scib.me.nmi(adata = adata, group1 = "tech",  group2 = "celltype")
        ari = scib.me.ari(adata = adata, group1 = "tech",  group2 = "celltype")
        
        # visualization
        scvi_vis(adata, vae, fig_path)

        int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                             "scvi", "pcr", pcr, fig_path, n_latent]
        int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                             "scvi", "nmi", nmi, fig_path, n_latent]
        int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                             "scvi", "ari", ari, fig_path, n_latent]
    print(int_metrics)
    int_metrics.to_csv("integration_results/scanorama_raw.csv")

## Perform integration for simulated data, integrated by scanorma, evaluated by three metrics

In [55]:
ad_file_paths = glob.glob("../../results/simulation2/test/derived_data/human_pancreas_norm_complexBatch/*/*h5ad")

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
int_metrics = pd.DataFrame(columns = ["ad", "dataset", "c_type", "n_cell", "cell_times", "seq_depth",
                                      "method", "metric", "evaluation"])

In [57]:
for path in ad_file_paths:
    adata = sc.read(path)
    res = scib.integration.scanorama(adata, batch = 'tech')
    c_type, n_cell, cell_times, seq_depth = path2para(path)
    pcr = scib.me.pcr_comparison(adata_pre = adata, adata_post = res, covariate = "celltype")
    nmi = scib.me.nmi(adata = res, group1 = "tech",  group2 = "celltype")
    ari = scib.me.ari(adata = res, group1 = "tech",  group2 = "celltype")

    int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                         "scanorama", "pcr", pcr]
    int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                         "scanorama", "nmi", nmi]
    int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                         "scanorama", "ari", ari]


Found 19084 genes among all datasets
[[0.         0.2716763  0.13571429 0.17919075 0.1734104 ]
 [0.         0.         0.25       0.25182482 0.24320652]
 [0.         0.         0.         0.23571429 0.24285714]
 [0.         0.         0.         0.         0.24635036]
 [0.         0.         0.         0.         0.        ]]
Processing datasets (0, 1)
Processing datasets (1, 3)
Processing datasets (1, 2)
Processing datasets (3, 4)
Processing datasets (1, 4)
Processing datasets (2, 4)
Processing datasets (2, 3)
Processing datasets (0, 3)
Processing datasets (0, 4)
Processing datasets (0, 2)
Variance contribution increased after integration!
Setting PCR comparison score to 0.
Found 19084 genes among all datasets
[[0.         1.         1.         1.         1.        ]
 [0.         0.         0.94915254 0.96       0.96875   ]
 [0.         0.         0.         0.96       0.94791667]
 [0.         0.         0.         0.         0.92      ]
 [0.         0.         0.         0.         0

Processing datasets (0, 4)
Processing datasets (0, 3)
Processing datasets (0, 2)
Processing datasets (0, 1)
Processing datasets (1, 4)
Processing datasets (2, 3)
Processing datasets (1, 3)
Found 19084 genes among all datasets
[[0.         0.2716763  0.13571429 0.17919075 0.1734104 ]
 [0.         0.         0.25       0.25182482 0.24320652]
 [0.         0.         0.         0.23571429 0.24285714]
 [0.         0.         0.         0.         0.24635036]
 [0.         0.         0.         0.         0.        ]]
Processing datasets (0, 1)
Processing datasets (1, 3)
Processing datasets (1, 2)
Processing datasets (3, 4)
Processing datasets (1, 4)
Processing datasets (2, 4)
Processing datasets (2, 3)
Processing datasets (0, 3)
Processing datasets (0, 4)
Processing datasets (0, 2)
Variance contribution increased after integration!
Setting PCR comparison score to 0.


In [58]:
int_metrics

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ad,dataset,c_type,n_cell,cell_times,seq_depth,method,metric,evaluation
0,path,human_pancreas_norm_complexBatch,alpha,2,0,1000,scanorama,pcr,0
1,path,human_pancreas_norm_complexBatch,alpha,2,0,1000,scanorama,nmi,0.0
2,path,human_pancreas_norm_complexBatch,alpha,2,0,1000,scanorama,ari,0.0
3,path,human_pancreas_norm_complexBatch,delta,2,0,5000,scanorama,pcr,0.606383
4,path,human_pancreas_norm_complexBatch,delta,2,0,5000,scanorama,nmi,0.0
5,path,human_pancreas_norm_complexBatch,delta,2,0,5000,scanorama,ari,0.0
6,path,human_pancreas_norm_complexBatch,activated_stellate,5,0,10000,scanorama,pcr,0.288187
7,path,human_pancreas_norm_complexBatch,activated_stellate,5,0,10000,scanorama,nmi,0.0
8,path,human_pancreas_norm_complexBatch,activated_stellate,5,0,10000,scanorama,ari,0.0
9,path,human_pancreas_norm_complexBatch,activated_stellate,0,0,5000,scanorama,pcr,0.655347


In [60]:
int_metrics
#int_metrics.to_csv("scanorama.csv")
int_metrics.to_csv("scanorama_sim2.csv")

In [53]:
adata = sc.read("../../data/raw_data/human_pancreas_norm_complexBatch.h5ad")
res = scib.integration.scanorama(adata, batch = 'tech')

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Found 19093 genes among all datasets
[[0.00000000e+00 6.01312910e-01 3.13479624e-02 8.86454183e-02
  1.49199793e-01 1.69322709e-02 1.99203187e-03 0.00000000e+00
  1.19521912e-02]
 [0.00000000e+00 0.00000000e+00 1.63009404e-01 4.29321663e-01
  1.34228188e-02 2.32018561e-03 4.37636761e-04 0.00000000e+00
  6.70241287e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.08150470e-01
  4.70219436e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.04388715e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.85045948e-02 3.01624130e-02 0.00000000e+00 4.60475825e-03
  2.80831099e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.83178654e-01 6.53588023e-01 5.04221028e-01
  6.70241287e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 4.42575406e-01 6.77494200e-01
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 4.

In [54]:
nmi = scib.me.nmi(adata = res, group1 = "tech",  group2 = "celltype")
ari = scib.me.ari(adata = res, group1 = "tech",  group2 = "celltype")
print(nmi, ari)

0.07837369020085136 0.028379073573485902


/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## multiple cells

In [62]:
ad_file_paths = glob.glob("../../results/simulation2/test/derived_data/human_pancreas_norm_complexBatch/multiple*/*h5ad")
path = ad_file_paths[0]
adata = sc.read(path)

In [63]:
res = scib.integration.scanorama(adata, batch = 'tech')

Found 19084 genes among all datasets
[[0.         0.35106383 0.15481172 0.22823985 0.22727273]
 [0.         0.         0.29811715 0.29319106 0.27530211]
 [0.         0.         0.         0.25732218 0.26046025]
 [0.         0.         0.         0.         0.29878049]
 [0.         0.         0.         0.         0.        ]]
Processing datasets (0, 1)
Processing datasets (3, 4)
Processing datasets (1, 2)
Processing datasets (1, 3)
Processing datasets (1, 4)
Processing datasets (2, 4)
Processing datasets (2, 3)
Processing datasets (0, 3)
Processing datasets (0, 4)
Processing datasets (0, 2)


In [64]:
pcr = scib.me.pcr_comparison(adata_pre = adata, adata_post = res, covariate = "celltype")
nmi = scib.me.nmi(adata = res, group1 = "tech",  group2 = "celltype")
ari = scib.me.ari(adata = res, group1 = "tech",  group2 = "celltype")

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Variance contribution increased after integration!
Setting PCR comparison score to 0.


In [65]:
print(pcr, nmi, ari)

0 0.08850386140290037 0.057387116302569235


/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
